In [1]:
# 1. buy ==1,
# 2. sell ==0

In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go

In [3]:
nifty = yf.download('^NSEI')['Adj Close']

## calculated drawdown
cum_max = nifty.cummax()
nifty = nifty.to_frame(name='price')  #Converted to dataframe first
nifty['cum_max'] = cum_max

drawdown = (nifty['price'] - nifty['cum_max'])/nifty['cum_max']*100
nifty['drawdown'] = drawdown
nifty = round(nifty,2)

[*********************100%***********************]  1 of 1 completed



In [4]:
trace = go.Scatter(
    x = nifty.index,
    y = nifty['drawdown'],
    mode = 'lines',
    name = 'Line'
)
layout = go.Layout(
    title = 'Nifty price',
    xaxis = dict(title = 'Date'),
    yaxis = dict(title = 'Price')
)
# fig object by passing both
fig = go.Figure(data=[trace], layout= layout)
fig.show()

In [5]:
sell_drawdown = 5  #5%
buy_bounce= 5      #5%

In [6]:
nifty['cum_min'] =  nifty['price'].cummin()

bounce_up = (nifty['price'] - nifty['cum_min'])/nifty['cum_min']*100
nifty['bounce_up'] = bounce_up
nifty = round(nifty,2)
nifty



## calculating entry and exit price
nifty['entry'] = np.where(nifty['bounce_up'] > buy_bounce, 1,0)  # buy = 1
nifty['exit']   = np.where(nifty['drawdown'] < - (sell_drawdown), 1,0)   # sell  =1



## position
nifty['position'] = 1
for i in range(len(nifty)):
    nifty['position'] = np.where(nifty['position'].shift() ==1,
                        np.where(nifty['exit'] ==1, 0, nifty['position'].shift()),
                        np.where(nifty['entry'] ==1,1,nifty['position'].shift() ))

nifty = nifty.iloc[2:].copy()
nifty['position'] = nifty['position'].astype(int)

## entry price
nifty['entry_price'] = nifty['price']
for i in range(1,len(nifty)): # skipping first
    if nifty['position'].iloc[i] == 1:
        if nifty['position'].shift().iloc[i] == 0:
            nifty['entry_price'].iloc[i] = nifty['price'].iloc[i]
        else:
            nifty['entry_price'].iloc[i] = nifty['entry_price'].shift().iloc[i]
    else:
        nifty['entry_price'].iloc[i] = 0

<ipython-input-6-11bcbdeb5967>:34: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-6-11bcbdeb5967>:36: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-6-11bcbdeb5967>:32: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [7]:
nifty['exit_price'] = 0
for i in range(len(nifty)):
    if (nifty['position'].shift().iloc[i] ==1) and (nifty['position'].iloc[i] ==0) :
        nifty['exit_price'].iloc[i] = nifty['price'].iloc[i]

    else:
        nifty['exit_price'].iloc[i] = 0

## PnL and Pnl_pct
nifty['PnL'] = np.where(nifty['exit_price'] !=0, nifty['exit_price']-nifty['entry_price'].shift(),0)
nifty['PnL_pct'] = (nifty['PnL']/nifty['entry_price'].shift())*100

df2 = nifty[['entry_price', 'exit_price', 'PnL', 'PnL_pct']]
df2
df2 = df2.reset_index()

<ipython-input-7-9365fe545f49>:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-7-9365fe545f49>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [8]:
df2['rough'] = np.where(df2['entry_price'] == df2['entry_price'].shift(),0,1)
df2['Date'] = df2['Date'].astype(str)

df2['Date2'] = np.where(
    ((df2['rough'] == 1) & (df2['entry_price'] != 0)), df2['Date'],
    np.where((df2['rough'] == 1) & (df2['exit_price'] != 0), df2['Date'], 0)
)

In [9]:
df2['Date2'] = df2['Date2'].replace(0,None)
df2['Date2'] = df2['Date2'].fillna(method='ffill')
df2.drop('rough', axis=1,inplace=True)
df2

,Date,entry_price,exit_price,PnL,PnL_pct,Date2
0,2007-09-19,4732.35,0.0,0.0,NaN,2007-09-19
1,2007-09-20,4732.35,0.0,0.0,0.0,2007-09-19
2,2007-09-21,4732.35,0.0,0.0,0.0,2007-09-19
3,2007-09-24,4732.35,0.0,0.0,0.0,2007-09-19
4,2007-09-25,4732.35,0.0,0.0,0.0,2007-09-19
...,...,...,...,...,...,...
3867,2023-07-03,17915.05,0.0,0.0,0.0,2023-04-27
3868,2023-07-04,17915.05,0.0,0.0,0.0,2023-04-27
3869,2023-07-05,17915.05,0.0,0.0,0.0,2023-04-27
3870,2023-07-06,17915.05,0.0,0.0,0.0,2023-04-27


In [10]:
df2['Date2'] = df2['Date2'].shift()
df2['entry_price_shifted'] = df2['entry_price'].shift()

In [11]:
df2.rename(columns={'Date2': 'entry_date'}, inplace=True)
df2.rename(columns={'Date': 'exit_date'}, inplace=True)

In [12]:
df3 = df2[['entry_date', 'exit_date', 'entry_price_shifted','PnL', 'PnL_pct']]
df3 = df3[df3['PnL'] !=0]